# Aggregator

An aggregator advertises and creates beacon cohorts our of a set of DID controllers. Then coordinates the aggregation of DID updates from their respective controllers.

The high level steps required for aggregation are as follows:

- A. Create a DID, already bound to an aggregator (Controller)
- **B. Aggregator creates cohort (Aggregator)**
- C. Update that DID document (Controller)
- **D. Aggregate Updates (Aggregator)**
- E. Resolve Updated DID Document (Resolver)

Steps A and B are two halves of an interactive protocol in which a DID controller joins a beacon cohort and includes the resulting beacon address in an offline DID they are creating. Similarly C and D are two halves of an interactive protocol whereby a DID controller submits an update to the Aggregator, who aggregates it along with other updates from cohort participants, requests authorization and broadcasts the aggregated update through a bitcoin spend from the beacon address.

**The communication protocols required to facilitate interaction between an aggregator and a DID controller still need to be defined (Our initial hope is to leverage NOSTR). For these notebooks we use the Copy + Paste protocol. You the notebook runner will be told when to go to different notebooks, execute cells and copy across payloads that can be used in other notebooks.**


This notebook implements the aggregator halves these flows (B and D in bold). It should be run alongside the notebooks for [Alexis](./Alexis.ipynb) and [Satoshi](./Satoshi.ipynp) who will be the cohort participants of a 2-of-2 beacon cohort.

In [1]:
from termcolor import colored
from buidl.ecc import S256Point

# B. Aggregator creates cohort

Creating a beacon cohort requires the following five steps:

1. Define Beacon Parameters (package into advertisement)
2. Announce/Publicize beacon advertisement
3. Collect beacon opt-ins
4. Create beacon address
5. Announce beacon cohort set (to members)
6. Collect cohort participant DIDs


<div id="define_beacon"/>
    
## B1 Define Beacon Parameters

The parameters of a beacon need to be specified by the aggregator. These include:

- Advert ID: Uniquely identifies the advert within the context of the aggregator
- Frequency & Latency Requirements: How often the beacon will publish updates and how fast participants are required to respond.
- Cohort Size: Number of participants, or min or max
- Beacon Type: Whether the beacon is going to use the CID or SparseMerkleTree method to aggegate updates
- Financials: 
    - Subscription model
    - Pay-per-update
- Communication Protocols
    - How does the aggregator reach contributors (for when/what)
    - How do contributors reach the aggregator (for when/what)
- How do you confirm delivery?
- Aggregator DID: Can be used to for establish secure communications
- Signature: Signature over beacon payload from verification method in aggregators DID


**Question:** Should the beacon advert be a VC?

### B1.1 Define Beacon Parameters

In [2]:


beacon_advert = {
    "id": "SOMEBEACONID",
    "type": "SMTAggregatorBeacon",
    "cohort_size": 2,
    "btc_network": "signet",
    # TODO: probably better in seconds. Or maybe bitcoin blocks
    "frequency": "1 per month",
    # TODO: DID based return serviceEndpoint
    "return_address": "SMT_Aggregator => Copy + Paste"
    
}

### B1.2 Sign Beacon Advert

TODO: Aggregator needs a DID, then need to sign the beacon advert using this DID. Maybe as a VC?

In [3]:
## TODO: Advertisement Signing





## B2. Announce / Publicize Beacon Advert

Use the Copy+Paste protocol to copy the beacon advert to all participating DID controllers. In this case [Alexis](./Alexis.ipynb) and [Satoshi](./Satoshi.ipynb).

More realistic mechanisms that aggregators might use include:

- Website (open to public)
- NOSTR (to known clients)
- Published in print
- Newsletter (email to known recipients)


In [4]:
print("\nCopy and paste beacon advert to all participants:\n")
print(f"beacon_advert={beacon_advert}")


Copy and paste beacon advert to all participants:

beacon_advert={'id': 'SOMEBEACONID', 'type': 'SMTAggregatorBeacon', 'cohort_size': 2, 'btc_network': 'signet', 'frequency': '1 per month', 'return_address': 'SMT_Aggregator => Copy + Paste'}


## B3 Collect Beacon OptIns

Beacon OptIns are payloads send from DID controllers who wish to join a beacon cohort. The aggregator recieves these payloads, verifies the signature is valid for the key in the payload and adds the key to a list of cohort participant public keys.

Paste in the outputs of step A2.4 in [Alexis](./Alexis.ipynb) and [Satoshi](./Satoshi.ipynb) notebooks

In [8]:
cohort_participants = [];

In [9]:
## From Alexis notebook
alexis_optin = {'advert_id': 'SOMEBEACONID', 'participant_pubkey': '03ec578937b7c1d975e5b05f5132673840aa6e9d3c5dc0f000f1ce432892ee77ad', 'return_interact': 'Alexis => Copy+Paste'}


In [10]:
alexis_hexkey = alexis_optin["participant_pubkey"]
alexis_sec_point = bytes.fromhex(alexis_hexkey)
alexis_pk = S256Point.parse(alexis_sec_point)

## TODO: Verify Signature - Request payload to have been signed

In [11]:
alexis_participant_data = {};
# Use hexkey as the ID?
alexis_participant_data["id"] = alexis_hexkey
alexis_participant_data["pk"] = alexis_pk
alexis_participant_data["dids"] = []
alexis_participant_data["return_interact"] = alexis_optin["return_interact"]

cohort_participants.append(alexis_participant_data)

In [13]:
## From Satoshi notebook
satoshi_optin = {'advert_id': 'SOMEBEACONID', 'participant_pubkey': '03f61957cc6e6b146b52a31198fe2749f3ac50362685cf9473ec44b084a143e016', 'return_interact': 'Satoshi => Copy+Paste'}


In [14]:
satoshi_hexkey = satoshi_optin["participant_pubkey"]
satoshi_sec_point = bytes.fromhex(satoshi_hexkey)
satoshi_pk = S256Point.parse(satoshi_sec_point)

## TODO: Verify Signature - Request payload to have been signed

In [15]:
satoshi_participant_data = {};
# Use hexkey as the ID?
satoshi_participant_data["id"] = satoshi_hexkey
satoshi_participant_data["pk"] = satoshi_pk
satoshi_participant_data["dids"] = []
satoshi_participant_data["return_interact"] = satoshi_optin["return_interact"]

cohort_participants.append(satoshi_participant_data)

## B4. Generate Beacon Address

In [16]:
from buidl.taproot import MuSigTapScript, TapRootMultiSig, P2PKTapScript

cohort_participant_pks = [participant["pk"] for participant in cohort_participants]
musig = MuSigTapScript(cohort_participant_pks)

### ADDITIONAL STEP BECAUSE OF BUIDL LIBRARY BUG

Basically the address must have a tweak. [See issue](https://github.com/buidl-bitcoin/buidl-python/issues/155)

In [19]:
tr_multisig = TapRootMultiSig(cohort_participant_pks, len(cohort_participant_pks))
internal_pubkey = tr_multisig.default_internal_pubkey
branch = tr_multisig.musig_tree()
tr_merkle_root = branch.hash()

network = beacon_advert["btc_network"]
p2tr_beacon_address = internal_pubkey.p2tr_address(tr_merkle_root,network=network)

In [20]:
print("Beacon Address Set : " + p2tr_beacon_address)

Beacon Address Set : tb1paq3mtqultmglkdk8sul2jmlsrx59vvnsgugwy7dekf8h5k0uk9dsvwa7uh


## B5. Announce beacon cohort set

### B5.1 Construct cohort set payload

Payload contains:

- Advert ID
- Participant cohort keys: List of participant public keys (as hex strings)
- Beacon Address: n-of-n p2tr beacon address
- Signature

In [21]:
hex_pks = [pk.sec().hex() for pk in cohort_participant_pks]

cohort_set_payload = {
    "advert_id": beacon_advert["id"],
    "participant_pubkeys": hex_pks,
    "beacon_address": p2tr_beacon_address
}

### B5.2 Sign cohort set payload

Cohort set payload should be signed by the DID of the aggregator provided in the beacon advert.

Another VC?

In [22]:
## TODO: signing



### B5.3 Send cohort set payload

Copy+Paste the below output into step A2.5 of the Alexis and Satoshi notebooks.

In [23]:
print("Copy+Paste the below output into step A2.5 of the Alexis and Satoshi notebooks.\n\n")
print(cohort_set_payload)

Copy+Paste the below output into step A2.5 of the Alexis and Satoshi notebooks.


{'advert_id': 'SOMEBEACONID', 'participant_pubkeys': ['03ec578937b7c1d975e5b05f5132673840aa6e9d3c5dc0f000f1ce432892ee77ad', '03f61957cc6e6b146b52a31198fe2749f3ac50362685cf9473ec44b084a143e016'], 'beacon_address': 'tb1paq3mtqultmglkdk8sul2jmlsrx59vvnsgugwy7dekf8h5k0uk9dsvwa7uh'}


### B5.4 Fund Beacon Address

The beacon address needs funds it can spend in order to broadcast beacon signals onto the network.

**This must be done outside of these notebooks.**

The funding transaction is https://mempool.space/signet/tx/6fcbf342e41fc9c5a18852436a3d25a850246de3617b1d47cb7b6586b77a077e

## B6. Collect cohort participant DIDs

This is likely an ongoing process. Participants of a cohort will be able to submit DIDs for which they require proofs of (non)inclusion for every signal the beacon broadcasts. 



### B6.1 Receive DID registration request from participants

Paste in the output from step A5.3 in [Alexis](./Alexis.ipynb) and [Satoshi](./Satoshi.ipynb) notebooks.

In [24]:
# From Alexis notebook
alexis_did_registration = {'beacon_address': 'tb1paq3mtqultmglkdk8sul2jmlsrx59vvnsgugwy7dekf8h5k0uk9dsvwa7uh', 'participant_pk': '03ec578937b7c1d975e5b05f5132673840aa6e9d3c5dc0f000f1ce432892ee77ad', 'did': 'did:btc1:x1n3xa88mx0e2h0n2sv70tq8f6xu8yr8afhs0ktr9gz8fsq5qzslcsuzsk0e'}

In [25]:
# From Satoshi notebook
satoshi_did_registration = {'beacon_address': 'tb1paq3mtqultmglkdk8sul2jmlsrx59vvnsgugwy7dekf8h5k0uk9dsvwa7uh', 'participant_pk': '03f61957cc6e6b146b52a31198fe2749f3ac50362685cf9473ec44b084a143e016', 'did': 'did:btc1:x1vqvraf5jftkqfz74wyevyq38xr75vavtwcq69kpgxgwdm9aa0v4srhv6vr'}

### B6.2 Verify DID registration request

- Check beacon address matches one of the aggregators beacons
- Check participant_pk is a participant key of the beacon
- Verify signature (TODO)

In [26]:
assert(alexis_did_registration["beacon_address"] == p2tr_beacon_address)
assert(satoshi_did_registration["beacon_address"] == p2tr_beacon_address)


In [27]:
participant_ids = [participant["id"] for participant in cohort_participants]

alexis_id = alexis_did_registration["participant_pk"]
satoshi_id = satoshi_did_registration["participant_pk"]

assert(alexis_id in participant_ids)
assert(satoshi_id in participant_ids)

In [28]:
## TODO: verify signatures on did registration payloads



### B6.3 Add DID to cohort participant data

This would be turned into a function

In [29]:
for participant in cohort_participants:
    participant_id = participant["id"]
    if (participant_id == alexis_id):
        alexis_did = alexis_did_registration["did"]
        participant["dids"].append(alexis_did)
    elif (participant_id == satoshi_id):
        satoshi_did = satoshi_did_registration["did"]
        participant["dids"].append(satoshi_did)


In [30]:
cohort_participants

[{'id': '03ec578937b7c1d975e5b05f5132673840aa6e9d3c5dc0f000f1ce432892ee77ad',
  'pk': S256Point(03ec578937b7c1d975e5b05f5132673840aa6e9d3c5dc0f000f1ce432892ee77ad),
  'dids': ['did:btc1:x1n3xa88mx0e2h0n2sv70tq8f6xu8yr8afhs0ktr9gz8fsq5qzslcsuzsk0e'],
  'return_interact': 'Alexis => Copy+Paste'},
 {'id': '03f61957cc6e6b146b52a31198fe2749f3ac50362685cf9473ec44b084a143e016',
  'pk': S256Point(03f61957cc6e6b146b52a31198fe2749f3ac50362685cf9473ec44b084a143e016),
  'dids': ['did:btc1:x1vqvraf5jftkqfz74wyevyq38xr75vavtwcq69kpgxgwdm9aa0v4srhv6vr'],
  'return_interact': 'Satoshi => Copy+Paste'}]

## Step B Aggregator Creates Beacon Cohort completed.

The aggregator now awaits DID update invocation payloads from beacon participants, that it will coordinate the aggregation of.

# D. Aggregate Updates



## D1. Collect cohort participant DID update invocations


In these notebooks we only aggregate a single update for simplicity.

### D1.1 Receive DID update payloads from cohort participants

Paste the output from step C4.4 of the [Alexis](./Alexis.ipynb) notebook. 


In [31]:


# Copy from step C4.4 of the Alexis notebook
alexis_update_payload = {'beacon_address': 'tb1paq3mtqultmglkdk8sul2jmlsrx59vvnsgugwy7dekf8h5k0uk9dsvwa7uh', 'invocation': {'@context': ['https://w3id.org/security/v2', 'https://w3id.org/zcap/v1', 'https://w3id.org/json-ld-patch/v1'], 'patch': [{'op': 'add', 'path': '/service/4', 'value': {'id': '#linked-domain', 'type': 'LinkedDomains', 'serviceEndpoint': 'https://contact-me.com'}}], 'sourceHash': '5wBb48EzdLf6us9vGNY97SAPaHUAeVyfPb25pSL7XHTU', 'targetHash': '4yK9dCraTdummpEderam5WDmNuMQAScDy6tt5mLPoD1K', 'targetVersionId': 2, 'proof': {'type': 'SchnorrSecp256k12024', 'created': '2024-01-04T17:59:51Z', 'verificationMethod': '#initialKey', 'proofPurpose': 'capabilityInvocation', 'capability': 'urn:zcap:root:did%3Abtc1%3Ax1n3xa88mx0e2h0n2sv70tq8f6xu8yr8afhs0ktr9gz8fsq5qzslcsuzsk0e', 'invocationTarget': 'did:btc1:x1n3xa88mx0e2h0n2sv70tq8f6xu8yr8afhs0ktr9gz8fsq5qzslcsuzsk0e', 'capabilityAction': 'write', 'proofValue': 'z381yXYmxU8NudZ4HXY56DfMN6zfD8syvWcRXzT9xD9uYoQToo8QsXD7ahM3gXTzuay5WJbqTswt2BKaGWYn2hHhVFKJLXaDz'}}, 'did': 'did:btc1:x1n3xa88mx0e2h0n2sv70tq8f6xu8yr8afhs0ktr9gz8fsq5qzslcsuzsk0e', 'participant_pk': '03ec578937b7c1d975e5b05f5132673840aa6e9d3c5dc0f000f1ce432892ee77ad'}

### D1.2 Verify DID update payload

- beacon_address matches one of the aggregators beacons
- payload is from a participant of the beacons cohort
- signature is valid
- DID update is for has previously been registered for participant (Does this matter, or is submitting an update for a new DID the same as registering?)

In [32]:
assert(alexis_update_payload["beacon_address"] == p2tr_beacon_address)

In [33]:
assert(alexis_update_payload["participant_pk"] in participant_ids)

In [34]:
## TODO: verify the signature.
## Need to have the signature first

In [35]:
update_did = alexis_update_payload["did"]
for participant in cohort_participants:
    if (participant["id"] == alexis_update_payload["participant_pk"]):
        participant_dids = participant["dids"]   
        assert(update_did in participant_dids)
        break;

### D1.3 Add DID update to invocation set

In [36]:
did_update_set = {}
update_invocation = alexis_update_payload["invocation"]
did_update_set[update_did] = update_invocation

In [37]:
did_update_set

{'did:btc1:x1n3xa88mx0e2h0n2sv70tq8f6xu8yr8afhs0ktr9gz8fsq5qzslcsuzsk0e': {'@context': ['https://w3id.org/security/v2',
   'https://w3id.org/zcap/v1',
   'https://w3id.org/json-ld-patch/v1'],
  'patch': [{'op': 'add',
    'path': '/service/4',
    'value': {'id': '#linked-domain',
     'type': 'LinkedDomains',
     'serviceEndpoint': 'https://contact-me.com'}}],
  'sourceHash': '5wBb48EzdLf6us9vGNY97SAPaHUAeVyfPb25pSL7XHTU',
  'targetHash': '4yK9dCraTdummpEderam5WDmNuMQAScDy6tt5mLPoD1K',
  'targetVersionId': 2,
  'proof': {'type': 'SchnorrSecp256k12024',
   'created': '2024-01-04T17:59:51Z',
   'verificationMethod': '#initialKey',
   'proofPurpose': 'capabilityInvocation',
   'capability': 'urn:zcap:root:did%3Abtc1%3Ax1n3xa88mx0e2h0n2sv70tq8f6xu8yr8afhs0ktr9gz8fsq5qzslcsuzsk0e',
   'invocationTarget': 'did:btc1:x1n3xa88mx0e2h0n2sv70tq8f6xu8yr8afhs0ktr9gz8fsq5qzslcsuzsk0e',
   'capabilityAction': 'write',
   'proofValue': 'z381yXYmxU8NudZ4HXY56DfMN6zfD8syvWcRXzT9xD9uYoQToo8QsXD7ahM3gXTz

## D2. Construct Sparse Merkle Tree (SMT) from DID update set

**Note: This step is done once Aggregator is happy that beacon update conditions met. Either certain number of updates submitted or amount of time has passed.**

In [38]:
from smt.tree import SparseMerkleTree
from smt.proof import verify_proof, SparseMerkleProof
from smt.utils import DEFAULTVALUE
from buidl.helper import str_to_bytes
import json

### D2.1 Initialize empty SMT

In [39]:
tree = SparseMerkleTree()

### D2.2 Add DID update invocations to set

For each update in set add a hash of the update invocation at the SMT leaf indexed by a hash of the DID.

The library sparse-merkle-tree handles the hashing, we just need to pass in byte strings to hash.

In [40]:
for key,value in did_update_set.items():
    did_bytes = str_to_bytes(key)
    print("KEY : " + key)
    update_string = json.dumps(value)
    did_update_bytes = str_to_bytes(update_string)
    tree.update(did_bytes, did_update_bytes)
    

KEY : did:btc1:x1n3xa88mx0e2h0n2sv70tq8f6xu8yr8afhs0ktr9gz8fsq5qzslcsuzsk0e


### D2.3 Calculate SMT root

In [41]:
smt_hex_root = tree.root_as_hex()
smt_hex_root

'0xd62e03e77d876ec16d775a4a4b31dbb3e785d439472c9b99d798d7eb7a9f0db1'

### D2.4 Create (non)inclusion proofs for each DID registered by cohort participants

In [42]:
import pickle
from smt.proof import verify_proof, SparseMerkleProof

for participant in cohort_participants:
    proofs = {}
    smt_root = tree.root_as_bytes()
    # return_interact = participant["return_interact"]
    dids = participant["dids"]
    for did in dids:
        did_bytes = str_to_bytes(did)
        proof = tree.prove(did_bytes)
        proof_bytes = pickle.dumps(proof.__dict__)

        proofs[did] = proof_bytes
    # TODO: should this be here or in a separate data structure
    participant["proofs"] = proofs
    print(f"\nProof for {did} : \n{proofs}\n")
    


Proof for did:btc1:x1n3xa88mx0e2h0n2sv70tq8f6xu8yr8afhs0ktr9gz8fsq5qzslcsuzsk0e : 
{'did:btc1:x1n3xa88mx0e2h0n2sv70tq8f6xu8yr8afhs0ktr9gz8fsq5qzslcsuzsk0e': b'\x80\x04\x95>\x00\x00\x00\x00\x00\x00\x00}\x94(\x8c\tsidenodes\x94]\x94\x8c\x17non_membership_leafdata\x94N\x8c\x0csibling_data\x94Nu.'}


Proof for did:btc1:x1vqvraf5jftkqfz74wyevyq38xr75vavtwcq69kpgxgwdm9aa0v4srhv6vr : 
{'did:btc1:x1vqvraf5jftkqfz74wyevyq38xr75vavtwcq69kpgxgwdm9aa0v4srhv6vr': b'\x80\x04\x95\x81\x00\x00\x00\x00\x00\x00\x00}\x94(\x8c\tsidenodes\x94]\x94\x8c\x17non_membership_leafdata\x94CA\x00+\xa1d;Q\xc1\xa2\x18\x1ea\x9a1\x08\xfc\xe1H\xea\x16\x9e4\x13[\xf0\x944\x93\xb2\xdc4~l)\xc5W\xa0\xf2\xb1\x88\x17P6V\x9e;B\xfe\x12\xc4\x06\xcd^\x05M\x02c\xed\xfee\xe8t\xb5\n\xe2\x14\x94\x8c\x0csibling_data\x94Nu.'}



## D3. Construct pending beacon signal

A pending beacon signal is an unsigned bitcoin transaction that spends from the beacon address and has a tx output at index 0 containing of the following format [OP_RETURN, smt_root]

### D3.1 Construct a transaction input (TxIn) that spends an output controlled by the beacon address

Funding transaction and associated ID were produced outside of the scope of these noteobooks. It can be seen on MemPool here - https://mempool.space/signet/tx/222b2223defe2b5cdb6261763753b8c755ce73a77d1bec596a6b308d8c60a2d4.

If these notebooks have been used to generate a different beacon address, a new funding transaction will have to be created.

In [43]:
from buidl.tx import Tx, TxIn, TxOut, SIGHASH_DEFAULT

# Transaction ID for transaction that sent funds to the beacon address
funding_tx_id = "b33dabe7c6ccbbfe27487692d1c9318fe4c478d68347acc6e1714f5066f97f36"

# TODO: Need to fund a beacon address
prev_tx = bytes.fromhex(funding_tx_id)  # Identifying funding tx
prev_index = 1 # Identify funding output index


tx_in = TxIn(prev_tx=prev_tx, prev_index=prev_index)

print("Tx Input satoshis: ",tx_in.value(network="signet"))

Tx Input satoshis:  10000


### D3.2 Construct a transaction output with 0 value and a ScriptPubKey of [OP_RETURN, smt_hash]

Note: OP_RETURN denoted as 0x6a hex value in bitcoin

In [44]:
smt_root_bytes = tree.root_as_bytes()

In [45]:
from buidl.script import ScriptPubKey



script_pubkey = ScriptPubKey([0x6a, smt_root_bytes])

beacon_signal_txout = TxOut(0, script_pubkey)

### D3.3 Construct a refunding transaction

This prevents all funds from the TxIn being sent to miners.

In [46]:

tx_fee = 350
refund_amount = tx_in.value(network="signet") - tx_fee
refund_out = TxOut.to_address(p2tr_beacon_address, refund_amount)

### D3.4 Construct the unsigned bitcoin transaction 

Transaction has a single input locked by the beacon address and two outputs.

**Note:** In the final protocol we will most likely use the partially signed bitcoin transaction (PSBT) data structure.

In [48]:
tx_ins = [tx_in]

tx_outs = [beacon_signal_txout,refund_out]

pending_beacon_signal = Tx(version=1, tx_ins=tx_ins, tx_outs=tx_outs, network=network,segwit=True)


print(pending_beacon_signal.serialize().hex())

010000000001010a785a1316ccebd9362d1c297e56ebdfc8e584fb3e32c851fb0ad9f574a708210100000000ffffffff020000000000000000226a20d62e03e77d876ec16d775a4a4b31dbb3e785d439472c9b99d798d7eb7a9f0db1b225000000000000225120e823b5839f5ed1fb36c7873ea96ff019a85632704710e279b9b24f7a59fcb15b0000000000


## D4. Send authorize beacon signal request to respective cohort participants.

Payload contains:

- Session ID: Unique identifier for this signing session, so that the payloads can be bound to the same signing session across the required communicaiton rounds.
- Beacon Address: Unique identifier for the beacon that is emmitting the signal
- Pending Beacon Signal: The PSBT that cohort participants are being asked to authorize. (**Note**: This is just a regular unsigned bitcoin transaction for the moment)
- Proofs: A data structure containing proofs of (non)inclusion respective to the dids each cohort participant has registered

In [49]:
# TODO: secure random generator?
session_id = "SOME SESSION ID"

## COMMON TO ALL PARTICIPANTS
authorize_beacon_signal_request = {
    "session_id": session_id,
    "beacon_address": p2tr_beacon_address,
    "pending_beacon_signal": pending_beacon_signal.serialize().hex(),
}

for participant in cohort_participants:
    authorize_beacon_signal_request["proofs"] = participant["proofs"]

    print("Communicate the authorize beacon signal request to the cohort participant using the specified return interact\n")
    print(participant["return_interact"])
    print("\n\n")
    print(authorize_beacon_signal_request)
    print("\n\n")
    

Communicate the authorize beacon signal request to the cohort participant using the specified return interact

Alexis => Copy+Paste



{'session_id': 'SOME SESSION ID', 'beacon_address': 'tb1paq3mtqultmglkdk8sul2jmlsrx59vvnsgugwy7dekf8h5k0uk9dsvwa7uh', 'pending_beacon_signal': '010000000001010a785a1316ccebd9362d1c297e56ebdfc8e584fb3e32c851fb0ad9f574a708210100000000ffffffff020000000000000000226a20d62e03e77d876ec16d775a4a4b31dbb3e785d439472c9b99d798d7eb7a9f0db1b225000000000000225120e823b5839f5ed1fb36c7873ea96ff019a85632704710e279b9b24f7a59fcb15b0000000000', 'proofs': {'did:btc1:x1n3xa88mx0e2h0n2sv70tq8f6xu8yr8afhs0ktr9gz8fsq5qzslcsuzsk0e': b'\x80\x04\x95>\x00\x00\x00\x00\x00\x00\x00}\x94(\x8c\tsidenodes\x94]\x94\x8c\x17non_membership_leafdata\x94N\x8c\x0csibling_data\x94Nu.'}}



Communicate the authorize beacon signal request to the cohort participant using the specified return interact

Satoshi => Copy+Paste



{'session_id': 'SOME SESSION ID', 'beacon_address': 'tb1paq3mtqultmglkdk8su

## D5. Aggregate MuSig2 Nonce

### D5.1 Collected public nonce points from cohort participants

**Must get fresh nonce points from each participant.**

Paste in nonce payloads for each participant from Step C4.5.2 of their respective notebooks.

In [51]:
alexis_nonce_payload = {'beacon_address': 'tb1paq3mtqultmglkdk8sul2jmlsrx59vvnsgugwy7dekf8h5k0uk9dsvwa7uh', 'session_id': 'SOME SESSION ID', 'nonce_points': ['025ff91f189127e3cbd8c5af15c33b0b51c2b0c835d56d89702b8d9e2bf7a205d8', '035bdf06226b9fbdb945fdca9050cab0faee7bedf8b5f1057759ab675df03be9a7']}

In [52]:
satoshi_nonce_payload = {'beacon_address': 'tb1paq3mtqultmglkdk8sul2jmlsrx59vvnsgugwy7dekf8h5k0uk9dsvwa7uh', 'session_id': 'SOME SESSION ID', 'nonce_points': ['0223bd55fe9e867f8429a6781f9d15aa46091983b431ef170d871e20e7ad972689', '023bb299b97e43397305578beef2ff01595818d763fab6dd9f8892f3c0382a3a2c']}

### D5.2 Verify nonce payloads

In [53]:
assert(p2tr_beacon_address == alexis_nonce_payload["beacon_address"])
assert(p2tr_beacon_address == satoshi_nonce_payload["beacon_address"])


In [54]:
assert(session_id == alexis_nonce_payload["session_id"])
assert(session_id == satoshi_nonce_payload["session_id"])


In [55]:
## TODO: Add and verify signatures over these payloads

### D5.3 Aggregate nonce points to create MuSig2 aggregated nonce

In [56]:
alexis_nonce_points =  [S256Point.parse(bytes.fromhex(hex_key)) for hex_key in alexis_nonce_payload['nonce_points']]

In [57]:
satoshi_nonce_points =  [S256Point.parse(bytes.fromhex(hex_key)) for hex_key in satoshi_nonce_payload['nonce_points']]

In [58]:
nonce_points = [alexis_nonce_points, satoshi_nonce_points]

nonce_sums = musig.nonce_sums(nonce_points)

In [59]:
nonce_sums

(S256Point(03f1846cc5047272ccca0827e0097d6a27bca46f53dcdc173c85b87caac92975b6),
 S256Point(02a88fc71c0dff088bb5cbc2c4dec2538a15a7c0f7a31e314ac3a0986bc253b4f6))

### D5.4 Send aggregated nonce points to all cohort participants

In [60]:
agg_nonce_hex = [nonce_point.sec().hex() for nonce_point in nonce_sums]
aggregated_nonce_payload = {
    "beacon_address": p2tr_beacon_address,
    "session_id": session_id,
    "agg_nonce": agg_nonce_hex
}

In [61]:
print("Copy+Paste the below output into step C4.5.3 of Alexis and Satoshi notebooks\n\n")
print(aggregated_nonce_payload)

Copy+Paste the below output into step C4.5.3 of Alexis and Satoshi notebooks


{'beacon_address': 'tb1paq3mtqultmglkdk8sul2jmlsrx59vvnsgugwy7dekf8h5k0uk9dsvwa7uh', 'session_id': 'SOME SESSION ID', 'agg_nonce': ['03f1846cc5047272ccca0827e0097d6a27bca46f53dcdc173c85b87caac92975b6', '02a88fc71c0dff088bb5cbc2c4dec2538a15a7c0f7a31e314ac3a0986bc253b4f6']}


## D6. Aggregate signatures and broadcast beacon signal

### D6.1 Collect signal authorizations from cohort participants

Paste output from step C4.8.2 of Alexis and Satoshi notebooks

In [62]:
partial_sigs = []

In [63]:
alexis_signal_authorization = {'beacon_address': 'tb1paq3mtqultmglkdk8sul2jmlsrx59vvnsgugwy7dekf8h5k0uk9dsvwa7uh', 'session_id': 'SOME SESSION ID', 'sig': 95548648078513855539470960323709747029731451725895585066518514894188892167143}

In [64]:
assert(p2tr_beacon_address == alexis_signal_authorization["beacon_address"])
assert(session_id == alexis_signal_authorization["session_id"])

In [65]:
partial_sigs.append(alexis_signal_authorization["sig"])

In [66]:
satoshi_signal_authorization = {'beacon_address': 'tb1paq3mtqultmglkdk8sul2jmlsrx59vvnsgugwy7dekf8h5k0uk9dsvwa7uh', 'session_id': 'SOME SESSION ID', 'sig': 89735577659157098147155861602191867711227124426401672005718564010100147502759}

In [67]:
assert(p2tr_beacon_address == satoshi_signal_authorization["beacon_address"])
assert(session_id == satoshi_signal_authorization["session_id"])

In [68]:
partial_sigs.append(satoshi_signal_authorization["sig"])

In [69]:
partial_sigs

[95548648078513855539470960323709747029731451725895585066518514894188892167143,
 89735577659157098147155861602191867711227124426401672005718564010100147502759]

### D6.2 Aggregate partial signatures

In [70]:
## Check have recieved a partial sig from each participant
assert(len(partial_sigs) == len(cohort_participants))

In [71]:
sig_sum = 0;
for partial_sig in partial_sigs:
    sig_sum += partial_sig

### D6.3 Generate MuSig2 r value from aggregated nonce and btc tx sig hash

This r value is the same one generated by cohort participants in step C4.6.2

In [72]:
input_index = 0
sig_hash = pending_beacon_signal.sig_hash(input_index, SIGHASH_DEFAULT)

In [73]:
r = musig.compute_r(nonce_sums, sig_hash)

### D6.4 Finalize btc transaction

In [74]:
schnorr = musig.get_signature(sig_sum, r, sig_hash, tr_merkle_root)

In [75]:
tx_in_to_finalize = pending_beacon_signal.tx_ins[0]
tx_in_to_finalize.finalize_p2tr_keypath(schnorr.serialize())

In [76]:
pending_beacon_signal.verify_input(0)

True

### D6.5 Broadcast btc transaction

In this example we will just past the output below into mempool.space at the url https://mempool.space/signet/tx/push.

Generally, it is recommended that aggregators run their own bitcoin nodes from which they will be able to broadcast btc transactions.

In [78]:
print(pending_beacon_signal.serialize().hex())

010000000001010a785a1316ccebd9362d1c297e56ebdfc8e584fb3e32c851fb0ad9f574a708210100000000ffffffff020000000000000000226a20d62e03e77d876ec16d775a4a4b31dbb3e785d439472c9b99d798d7eb7a9f0db1b225000000000000225120e823b5839f5ed1fb36c7873ea96ff019a85632704710e279b9b24f7a59fcb15b0140c6ce4125813f8a6cc8e2b4bd8abf24d5a40a8ab05a9c2c3e2c44f31afaf51ff6888210da0d68425d3b39283f7fcb97925252b91f8545bb15130e4e24a2fa300800000000


# Step D Aggregate Updates Complete!!



Note: the transaction ID will change every time you run through these notebooks. 

In [79]:
print(f"The broadcast beacon signal can be found at: https://mempool.space/{network}/tx/{pending_beacon_signal.id()}")

The broadcast beacon signal can be found at: https://mempool.space/signet/tx/b33dabe7c6ccbbfe27487692d1c9318fe4c478d68347acc6e1714f5066f97f36
